In [17]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import math

In [2]:
!pip install textwrap3

from textwrap3 import wrap

In [ ]:
!pip install nltk

import nltk

nltk.download('stopwords')

In [3]:
input_file_path = "movie_plots.csv"

df = pd.read_csv(input_file_path)
print (df.shape)
df.head()

(3192, 4)


,Unnamed: 0,title,url,plot
0,0,White Noise,https://en.wikipedia.org/wiki/White_Noise_(200...,\nJonathan Rivers is an architect and lives wi...
1,1,Coach Carter,https://en.wikipedia.org/wiki/Coach_Carter,"\nKen Carter lives in Richmond, California. He..."
2,2,Elektra,https://en.wikipedia.org/wiki/Elektra_(2005_film),"\nAfter being killed in Daredevil, Elektra Nat..."
3,3,Racing Stripes,https://en.wikipedia.org/wiki/Racing_Stripes,"\nDuring a thunderstorm, a traveling circus, C..."
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org/wiki/Tom_and_Jerry:_B...,\nTom (voiced by Bill Kopp) chases Jerry as us...


In [5]:
batmanbegins_df = df[df['title'].str.contains("Batman Begins")]
batmanbegins_df.head()

,Unnamed: 0,title,url,plot
65,65,Batman Begins,https://en.wikipedia.org/wiki/Batman_Begins,"\nIn Gotham City, a young Bruce Wayne falls do..."


In [ ]:
batmanbegins_plot = batmanbegins_df.iloc[0]['plot']
batmanbegins_plot

In [ ]:
for line in wrap(batmanbegins_plot, 140):
    print(line)

In [ ]:
word_list = re.findall(r"[\w']+", batmanbegins_plot)
print(word_list)

In [25]:
word_counts = Counter(word_list)
print(word_counts.most_common(10))

[('the', 32), ('a', 28), ('and', 28), ('to', 26), ('him', 16), ('Bruce', 14), ('of', 12), ('his', 12), ('Batman', 12), ('by', 11)]


In [32]:
stopwords = set(nltk.corpus.stopwords.words("english"))

In [ ]:
[w.lower() for w in word_list if w.lower() not in stopwords]

In [ ]:
word_tf = { k : v / len(word_list) for k, v in word_counts.items() }
word_tf

In [37]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sanke\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [43]:
sample_sentence = "Elon Musk in recent months often tweeted for Dodgecoin, but rarely for Bitcoin"

tokenized_words = word_tokenize(sample_sentence)
print(tokenized_words)

bigrams = ngrams(tokenized_words, 2)
bigrams = [ ' '.join(grams) for grams in bigrams]
print(bigrams)

trigrams = ngrams(tokenized_words, 3)
trigrams = [ ' '.join(grams) for grams in trigrams]
print(trigrams)

['Elon', 'Musk', 'in', 'recent', 'months', 'often', 'tweeted', 'for', 'Dodgecoin', ',', 'but', 'rarely', 'for', 'Bitcoin']
['Elon Musk', 'Musk in', 'in recent', 'recent months', 'months often', 'often tweeted', 'tweeted for', 'for Dodgecoin', 'Dodgecoin ,', ', but', 'but rarely', 'rarely for', 'for Bitcoin']
['Elon Musk in', 'Musk in recent', 'in recent months', 'recent months often', 'months often tweeted', 'often tweeted for', 'tweeted for Dodgecoin', 'for Dodgecoin ,', 'Dodgecoin , but', ', but rarely', 'but rarely for', 'rarely for Bitcoin']


In [44]:
def get_term_frequency(input):
    tokenized_words = word_tokenize(input)
    cleaned_words = [w.lower() for w in tokenized_words if w.lower() not in stopwords]
    word_counts = Counter(cleaned_words)
    return  { k : float("{:.5f}".format(v / len(tokenized_words))) for k, v in word_counts.items() }

In [ ]:
get_term_frequency(batmanbegins_plot)

In [46]:
df.head()

,Unnamed: 0,title,url,plot
0,0,White Noise,https://en.wikipedia.org/wiki/White_Noise_(200...,\nJonathan Rivers is an architect and lives wi...
1,1,Coach Carter,https://en.wikipedia.org/wiki/Coach_Carter,"\nKen Carter lives in Richmond, California. He..."
2,2,Elektra,https://en.wikipedia.org/wiki/Elektra_(2005_film),"\nAfter being killed in Daredevil, Elektra Nat..."
3,3,Racing Stripes,https://en.wikipedia.org/wiki/Racing_Stripes,"\nDuring a thunderstorm, a traveling circus, C..."
4,4,Tom and Jerry: Blast Off to Mars,https://en.wikipedia.org/wiki/Tom_and_Jerry:_B...,\nTom (voiced by Bill Kopp) chases Jerry as us...


In [47]:
df['TF'] = df.apply(lambda row: get_term_frequency(row['plot']), axis=1)

In [50]:
df.shape

(3192, 5)

In [52]:
for index, row in df.iterrows():
    print(list(row['TF'].keys())[:5])
    if index == 5:
        break

['jonathan', 'rivers', 'architect', 'lives', 'wife']
['ken', 'carter', 'lives', 'richmond', ',']
['killed', 'daredevil', ',', 'elektra', 'natchios']
['thunderstorm', ',', 'traveling', 'circus', 'sarano']
['tom', '(', 'voiced', 'bill', 'kopp']
['two', 'rebellious', 'children', ',', 'lindsey']


In [54]:
print(df.shape)
df.drop_duplicates(subset=['url'], inplace=True, keep=False)
print(df.shape)

(3178, 5)
(3178, 5)


In [55]:
test_word = "gotham"

doc_count = 0
for index, row in df.iterrows():
    if test_word in row['TF']:
        doc_count += 1
        print(f"Movie Name: {row['title']} {row['url']}")
print(f"Document Count: {doc_count}")
print(f"Total documents: {df.shape[0]}")

Movie Name: Batman Begins https://en.wikipedia.org/wiki/Batman_Begins
Movie Name: The Batman vs. Dracula https://en.wikipedia.org/wiki/The_Batman_vs._Dracula
Movie Name: The Dark Knight https://en.wikipedia.org/wiki/The_Dark_Knight
Movie Name: Suicide Squad https://en.wikipedia.org/wiki/Suicide_Squad_(film)
Movie Name: The Lego Batman Movie https://en.wikipedia.org/wiki/The_Lego_Batman_Movie
Movie Name: Joker https://en.wikipedia.org/wiki/Joker_(2019_film)
Movie Name: Birds of Prey https://en.wikipedia.org/wiki/Birds_of_Prey_(2020_film)
Document Count: 7
Total documents: 3178


In [62]:
def get_idf(word, df):
    doc_count = 0
    for index, row in df.iterrows():
        if word in row['TF']:
            doc_count += 1
    idf = math.log10((df.shape[0] + 1) / (doc_count + 1)) + 1
    return idf
    

In [63]:
get_idf("gotham", df)

3.5992005409228294

In [72]:
word_list_by_plot_dict = df['TF'].tolist()
word_list_by_plot = [list(x.keys()) for x in word_list_by_plot_dict]
word_list_by_plot = [list(set(sublist)) for sublist in word_list_by_plot]
word_list_all = [item for sublist in word_list_by_plot for item in sublist]
word_list_all_distinct = list(set(word_list_all))

print(f"Words: {len(word_list_all)}, Distinct Words: {len(word_list_all_distinct)}")

Words: 730054, Distinct Words: 51716


In [ ]:
idf_dict = {}
for word in word_list_all_distinct:
    idf_dict[word] = get_idf(word, df)

In [ ]:
!pip install -U scikit-learn

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [6]:
tfidfvectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
tfidfvectorizer.fit(df['plot'])

TfidfVectorizer(stop_words='english')

In [7]:
feature_names = tfidfvectorizer.get_feature_names()
print(len(feature_names))
print(feature_names[:10])

46302
['00', '000', '000th', '001', '003', '0054', '007', '04', '05', '060']


C:\Users\sanke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [9]:
print(tfidfvectorizer.idf_)
tfidfvectorizer.idf_.shape

[6.17834443 3.55932786 7.9701039  ... 8.37556901 8.37556901 8.37556901]


(46302,)

In [43]:
tfidf = tfidfvectorizer.transform(df['plot'])

In [44]:
tfidf.shape

(3192, 46302)

In [ ]:
tfidf_df = pd.DataFrame(tfidf.toarray(), columns=feature_names)
tfidf_df.head(20)

Extract Keywords

In [18]:
movie_df = df[df['title'].str.contains("V for Vendetta")]
movie_plot = movie_df.iloc[0]['plot']
movie_tfidf = tfidfvectorizer.transform([movie_plot])

In [32]:
movie_tfidf_array = movie_tfidf.toarray()
feature_array = np.array(feature_names)

print(feature_array)
print(movie_tfidf_array.flatten())

sorted_indices = np.argsort(movie_tfidf_array.flatten())[::-1]
print(sorted_indices)

feature_sorted = feature_array[sorted_indices]
print(feature_sorted)

movie_tfidf_sorted = movie_tfidf_array.flatten()[sorted_indices]
print(movie_tfidf_sorted)

['00' '000' '000th' ... 'пре' 'рођења' 'умро']
[0. 0. 0. ... 0. 0. 0.]
[14407 28642  9747 ... 30817 30816     0]
['evey' 'norsefire' 'creedy' ... 'pete' 'petar' '00']
[0.64167446 0.27500334 0.27500334 ... 0.         0.         0.        ]


In [34]:
batmanbegins_df = df[df['title'].str.contains("Batman Begins")]
batmanbegins_plot = batmanbegins_df.iloc[0]['plot']

In [36]:
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(batmanbegins_plot)
sentences_tfidf = tfidfvectorizer.transform(sentences).toarray()

In [ ]:
sentence_and_score_array = []
for tfidf, sentence in zip(sentences_tfidf, sentences):
    indices = np.nonzero(tfidf)[0]
    scores = tfidf[indices]
    length = len(sentence.split())
    avg_score = scores.sum() / length
    sentence_and_score_array.append((avg_score, sentence))

sorted_sentence_scores = sorted(sentence_and_score_array, key=lambda x: x[0], reverse=True)

sorted_sentence_scores


Similarity Search

In [45]:
index = batmanbegins_df.iloc[0][0]
index

65

In [46]:
search_vec = tfidf[index]
search_vec

<1x46302 sparse matrix of type '<class 'numpy.float64'>'
	with 286 stored elements in Compressed Sparse Row format>

In [48]:
from sklearn.metrics.pairwise import linear_kernel

cosine_similarities = linear_kernel(search_vec, tfidf).flatten()

cosine_similarities

array([0.020465  , 0.00910681, 0.00778492, ..., 0.0143591 , 0.01072224,
       0.00937173])

In [50]:
related_docs_indices = cosine_similarities.argsort()[::-1]
related_docs_indices = list(related_docs_indices)[:10]
print(related_docs_indices)

[65, 677, 2033, 128, 1942, 1860, 1983, 1747, 2371, 1275]


In [52]:
for i in related_docs_indices:
    print(f"{df.iloc[i]['title']}")

Batman Begins
The Dark Knight
The Lego Batman Movie
The Batman vs. Dracula
Batman: The Killing Joke
I Smile Back
I'm Not Ashamed
Road Hard
Crazy Rich Asians
A Dark Truth
